In [1]:
import numpy as np
import cv2
import tensorflow as tf
import os

In [32]:
class KNNClassifier:
  def __init__(self, topk):
    self.dataset = []
    self.classtoid = {}
    self.idtoclass = {}
    self.entries = []
    self.nextclassid = 0
    self.topk = topk

  def addExample(self, image, label):

    if label not in self.classtoid:
      self.classtoid[label] = self.nextclassid
      self.idtoclass[self.nextclassid] = label
      self.nextclassid += 1

    label = self.classtoid[label]
    self.entries.append(label)
    self.dataset.append(image.flatten())

  def similarities(self, image):
    return np.sqrt(np.mean((self.dataset-image)**2, axis = 1))

  def predict(self, image):
    image = image.flatten()
    if self.topk>len(list(self.classtoid.keys())):
      topk = len(list(self.classtoid.keys()))
    else:
      topk = self.topk

    rms = self.similarities(image)
    topindices = np.argpartition(rms, topk)[:topk]
    classesintopk = np.array(self.entries)[topindices]
    unique, count = np.unique(classesintopk, return_counts=True)
    sums = []

    if len(np.where(count==count.max())[0])!=1:
      topkrms = np.array(rms)[topindices]
      for i in unique:
        idx = np.where(i==classesintopk)
        rmssum = np.array(topkrms)[idx[0]].sum()
        sums.append(rmssum)
      minid = np.argmin(sums)
      pred = unique[minid]
    else:
      pred = unique[count.argmax()]

    return self.idtoclass[pred]
  def set_topk(self, topk):
        self.topk = topk


In [33]:
model = tf.keras.models.load_model("MobileNet.h5")
model = tf.keras.models.Model(model.input, model.get_layer("global_average_pooling2d_1").output)

In [34]:
dataset = "D:/datasets/dataset/"
knn = KNNClassifier(7)

classes = os.listdir(dataset)
print("Found {} classes".format(len(classes)))
for cls in classes:
  path = dataset + cls + '/'
  count = os.listdir(path)
  print("Found {} examples in {} class".format(len(count), cls))
  for name in count:
    img = cv2.imread(path + name)
    img = cv2.resize(img, (224,224))
    pred = model.predict(img[np.newaxis,:])
    knn.addExample(pred, cls)

Found 2 classes
Found 962 examples in cats class
Found 964 examples in dogs class


In [36]:
#try on class = cat
for i in range(1,16):
    knn.set_topk(i)

    true_cat_predictions = []
    path = "D:/datasets/test_dataset/"
    li = os.listdir(path + 'cats/')
    for name in li:
        img = cv2.imread("D:/datasets/test_dataset/cats/" + name)
        img = cv2.resize(img, (224,224))
        pred = model.predict(img[np.newaxis,:])
        p = knn.predict(pred)
        true_cat_predictions.append(p)
    #try on class = cat
    true_dog_predictions = []
    path = "D:/datasets/test_dataset/"
    li = os.listdir(path + 'dogs/')
    for name in li:
        img = cv2.imread("D:/datasets/test_dataset/dogs/" + name)
        img = cv2.resize(img, (224,224))
        pred = model.predict(img[np.newaxis,:])
        p = knn.predict(pred)
        true_dog_predictions.append(p)

    print("For k = {}".format(knn.topk))    
    print("For Cats:")
    truec = len(np.where(np.array(true_cat_predictions)=='cats')[0])
    print("{} from {} predicted successfully".format(true, len(true_cat_predictions)))
    print("For Dogs:")
    trued = len(np.where(np.array(true_dog_predictions)=='dogs')[0])
    print("{} from {} predicted successfully".format(true, len(true_dog_predictions)))
    print("Overall Accuracy: {}%\n".format(np.round((truec+trued)/(len(true_cat_predictions)+len(true_dog_predictions))*100,2)))


For k = 1
For Cats:
21 from 36 predicted successfully
For Dogs:
21 from 36 predicted successfully
Overall Accuracy: 61.11%

For k = 2
For Cats:
21 from 36 predicted successfully
For Dogs:
21 from 36 predicted successfully
Overall Accuracy: 61.11%

For k = 3
For Cats:
21 from 36 predicted successfully
For Dogs:
21 from 36 predicted successfully
Overall Accuracy: 61.11%

For k = 4
For Cats:
21 from 36 predicted successfully
For Dogs:
21 from 36 predicted successfully
Overall Accuracy: 61.11%

For k = 5
For Cats:
21 from 36 predicted successfully
For Dogs:
21 from 36 predicted successfully
Overall Accuracy: 61.11%

For k = 6
For Cats:
21 from 36 predicted successfully
For Dogs:
21 from 36 predicted successfully
Overall Accuracy: 61.11%

For k = 7
For Cats:
21 from 36 predicted successfully
For Dogs:
21 from 36 predicted successfully
Overall Accuracy: 61.11%

For k = 8
For Cats:
21 from 36 predicted successfully
For Dogs:
21 from 36 predicted successfully
Overall Accuracy: 61.11%

For k = 

In [37]:

    knn.set_topk(100)

    true_cat_predictions = []
    path = "D:/datasets/test_dataset/"
    li = os.listdir(path + 'cats/')
    for name in li:
        img = cv2.imread("D:/datasets/test_dataset/cats/" + name)
        img = cv2.resize(img, (224,224))
        pred = model.predict(img[np.newaxis,:])
        p = knn.predict(pred)
        true_cat_predictions.append(p)
    #try on class = cat
    true_dog_predictions = []
    path = "D:/datasets/test_dataset/"
    li = os.listdir(path + 'dogs/')
    for name in li:
        img = cv2.imread("D:/datasets/test_dataset/dogs/" + name)
        img = cv2.resize(img, (224,224))
        pred = model.predict(img[np.newaxis,:])
        p = knn.predict(pred)
        true_dog_predictions.append(p)

    print("For k = {}".format(knn.topk))    
    print("For Cats:")
    truec = len(np.where(np.array(true_cat_predictions)=='cats')[0])
    print("{} from {} predicted successfully".format(true, len(true_cat_predictions)))
    print("For Dogs:")
    trued = len(np.where(np.array(true_dog_predictions)=='dogs')[0])
    print("{} from {} predicted successfully".format(true, len(true_dog_predictions)))
    print("Overall Accuracy: {}%\n".format(np.round((truec+trued)/(len(true_cat_predictions)+len(true_dog_predictions))*100,2)))


For k = 100
For Cats:
21 from 36 predicted successfully
For Dogs:
21 from 36 predicted successfully
Overall Accuracy: 61.11%

